#### As bibliotecas abaixo são necessárias para executar o notebook
pip install pyyaml

pip install nltk

pip install beautifulsoup

#pip install pypdf2  #para python >=3.4

pip install pdfminer.six #para python >=3.4


### Scraping no site do STF
O scraping no site do STF deverá ser feito através de um processo automático, mas será necessário a utilização de URLs
definidas através da pesquisa. Inclusive de termos que possam cobrir a maioria dos temas relativos à justiça do trabalho. 

#### Exemplo de uma URL que pode ser utilizada para recuperar uma página de pesquisa de acórdãos (em HTML)
http://www.stf.jus.br/portal/jurisprudencia/listarJurisprudencia.asp?s1=%28JUSTICA+ADJ2+TRABALHO+E+COMPET%24+E+ACIDENTE+ADJ2+TRABALHO%29&base=baseAcordaos&url=http://tinyurl.com/l2448cn

Essa pesquisa faz parte da seção de Pesquisas Prontas do site do STF, relacionado ao tema Indenização por Acidente do Trabalho e a Competência da Justiça do Trabalho


In [2]:
from urllib import request
from bs4 import BeautifulSoup

url = "http://www.stf.jus.br/portal/jurisprudencia/listarJurisprudencia.asp?s1=%28459510%2ENUME%2E+OU+459510%2EACMS%2E%29&base=baseAcordaos&url=http://tinyurl.com/jmkj3ae"
paginahtml = request.urlopen(url).read().decode("iso-8859-1")
txthtml = BeautifulSoup(paginahtml, 'lxml')
txthtml.text

'\n\n\n\n\n\r\n    Pesquisa de Jurisprudência :: STF - Supremo Tribunal Federal\r\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n(function(){\r\n\t\tvar qs,js,q,s,d=document,gi=d.getElementById,ce=d.createElement,gt=d.getElementsByTagName,id=\'typef_orm\',b=\'https://s3-eu-west-1.amazonaws.com/share.typeform.com/\';\r\n\t\tif(!gi.call(d,id)){js=ce.call(d,\'script\');js.id=id;js.src=b+\'share.js\';q=gt.call(d,\'script\')[0];q.parentNode.insertBefore(js,q)}})()\r\n\t\n\r\n\t\t\t\t\r\n\t\t$(document).ready(function() {\r\n\t\t\t$("#fechar").click(ocultarBarra);\r\n\t\t\t});\r\n \r\n            function ocultarBarra(){\r\n                $("#barraSuperior").hide();\r\n            }\r\n\t\t\r\n\t\n\n\n\n\n\n\n\n\n\n\nPágina principal\n\n\n\n\r\n\t            Brasília, 30 de abril de 2017 - 02:11\r\n        \n\n\n\n\nEspaço do Servidor\n\n\n | \n\n\nENGLISH\n\n\n | \n\n\nESPAÑOL\n\n\nMapa do Portal\n\n\n\n\n\ntermo\n\n\n\nInforme o termo\n\n\n\n\n\nPrincipal\n\n\nSobre o STF\n\n\nCon

### Os dados HTML estão sem nenhum tratamento ou limpeza
Para o teste vamos recuperar apenas os links e entre os links vamos separar apenas os links dos arquivos PDF

In [3]:
# recupera todos os links
todos_links = txthtml.find_all('a')
base_link = "http://www.stf.jus.br/portal"

# separa apenas os links de acordãos
links_acordao = []
for l in todos_links:
    if "inteiro" in l.attrs['href'] and "idDocumento=" in l.attrs['href']:   # nem todos os links vem corretos
        l_href = str(l.attrs['href'])[2:-2]
        id_doc = l_href[l_href.find('?')+1:]        
        links_acordao.append((l.text+" "+id_doc, base_link+l_href))

for l in links_acordao:
    print('{0} - {1}'.format(l[0], l[1]))



Inteiro Teor idDocumento=10710211 - http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=10710211


### Agora separamos cada um dos processos separados pelas divs 
Essa operação e todas as outras de scraping são fortemente dependentes do layout do site do STF. Caso haja, alguma mudança no site ela terá que ser refeita.

In [4]:
divsAcordaos = txthtml.find_all("div", {"class": "abasAcompanhamento"})

acordaoqlq = divsAcordaos[0].text
print(acordaoqlq)





Acompanhamento Processual
Inteiro Teor

DJ/DJe 
Ementa sem Formatação




RE 459510 / MT - MATO GROSSO
				





            
            
			RECURSO EXTRAORDINÁRIO
            Relator(a):&nbsp Min. CEZAR PELUSORelator(a) p/ Acórdão:&nbsp Min. DIAS TOFFOLIJulgamento:&nbsp 	26/11/2015&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp Órgão Julgador:&nbsp Tribunal Pleno


Publicação
ACÓRDÃO ELETRÔNICO
DJe-067  DIVULG 11-04-2016  PUBLIC 12-04-2016
Parte(s)
RECTE.(S)  : MINISTÉRIO PÚBLICO FEDERAL
PROC.(A/S)(ES) : PROCURADOR-GERAL DA REPÚBLICA
RECDO.(A/S)  : GILVAN JOSÉ GARAFFA
RECDO.(A/S)  : LUCIANE FRANCIO GARAFFA
RECDO.(A/S)  : JOÃO MARIA BASSANI
RECDO.(A/S)  : HEITOR CLEMENTE
ADV.(A/S)  : ALDO MÁRIO DE FREITAS LOPES

Ementa
Ementa
    Recurso extraordinário. Constitucional. Penal. Processual Penal. Competência. Redução a condição análoga à de escravo. Conduta tipificada no art. 149 do Código Penal. Crime contra a organização do trabalho. Competência da Justiça Federal. Artigo
109, inc

### Separação de cada uma das partes
Alguns acórdãos possuem uma lista de indexação que talvez possa ser utilizada no processo de estruturação de dados.

In [5]:
aba = divsAcordaos[0]
print(aba)


<div class="abasAcompanhamento" style="*position:relative;*left:3px;">
<ul class="abas">
<li><a href="../processo/verProcessoAndamento.asp?numero=459510&amp;classe=RE&amp;codigoClasse=0&amp;origem=JUR&amp;recurso=0&amp;tipoJulgamento=M">Acompanhamento Processual</a></li>
<li><a href="../inteiroTeor/obterInteiroTeor.asp?idDocumento=10710211
">Inteiro Teor</a></li>
<!--&nbsp; / -->
<li><a href="../diarioJustica/listarDiarioJustica.asp?tipoPesquisaDJ=AP&amp;numero=459510&amp;classe=RE">DJ/DJe </a></li>
<li class="ultima"><a href="javascript:void(0);" onclick="abrePop('visualizarEmenta.asp?s1=000307747&amp;base=baseAcordaos',620,360);">Ementa sem Formatação</a></li>
</ul>
<div class="processosJurisprudenciaAcordaos">
<a href="javascript:void(0);" onclick="abrePop('listarJurisprudenciaDetalhe.asp?s1=000307747&amp;base=baseAcordaos',620,500);"><img alt="Salvar" src="imagem/botao_salvar.gif" style="position:relative;left:494px;top:-38px;margin-bottom:-55px;"/></a>
<a href="javascript:void(0);

In [8]:
indexacao = acordaoqlq[acordaoqlq.find("Indexação"):acordaoqlq.find("Legislação")]
idx = indexacao.split(',')
idx = idx[0].split(' ')[1:] + idx[1:]
idx


['VIDE',
 'EMENTA.\n-',
 'FUNDAMENTAÇÃO',
 'COMPLEMENTAR',
 ' MIN. JOAQUIM BARBOSA: NECESSIDADE',
 ' VERIFICAÇÃO',
 ' INTÉRPRETE',
 ' CASO CONCRETO',
 ' REFERÊNCIA',
 ' CRIME',
 ' ORGANIZAÇÃO',
 ' TRABALHO',
 ' DECORRÊNCIA',
 ' AUSÊNCIA',
 ' TAXATIVIDADE',
 ' DELITO',
 ' CÓDIGO PENAL',
 ' FUNDAMENTO',
 ' DOUTRINA',
 ' JURISPRUDÊNCIA.\n- FUNDAMENTAÇÃO COMPLEMENTAR',
 ' MIN. ROSA WEBER: DESNECESSIDADE',
 ' CONSTRANGIMENTO ILEGAL',
 ' COAÇÃO FÍSICA',
 ' OBJETIVO',
 ' CARACTERIZAÇÃO',
 ' CERCEAMENTO',
 ' LIBERDADE',
 ' FUNDAMENTO',
 ' POSSIBILIDADE',
 ' CERCEAMENTO',
 ' LIBERDADE',
 ' DIVERSIDADE',
 ' CONSTRANGIMENTO ILEGAL',
 ' INCLUSÃO',
 '\r\nCARÁTER ECONÔMICO. DESNECESSIDADE',
 ' COAÇÃO FÍSICA',
 ' LIBERDADE DE LOCOMOÇÃO',
 ' OBJETIVO',
 ' CONFIGURAÇÃO',
 ' CRIME',
 ' REDUÇÃO À CONDIÇÃO ANÁLOGA À DE ESCRAVO',
 ' DECORRÊNCIA',
 ' SUFICIÊNCIA',
 ' SUBMISSÃO',
 ' VÍTIMA',
 ' TRABALHO FORÇADO',
 ' JORNADA EXAUSTIVA',
 ' CONDIÇÃO DEGRADANTE DE\r\nTRABALHO',
 ' OBJETIVO',
 ' CARACTERIZAÇÃO D